<a href="https://colab.research.google.com/github/lokesh1197/predict_document_class/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%%script echo skipping

data_dir = "datathonindoml-2022"

# from google.colab import drive
# drive.mount('/content/drive')

# !unzip /content/drive/MyDrive/Kaggle\ Datathon/datathonindoml-2022.zip 
# data_dir = "/content"


skipping


In [26]:
%%script echo skipping
!pip install Pillow
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install joblib

skipping


In [38]:
from os.path import join, isfile
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import sklearn
import joblib

In [20]:
trainDir = join(data_dir, "train/train")
validationDir = join(data_dir, "validation/validation")

labels = pd.read_csv(join(data_dir, "train_labels.csv"))
labels.describe()

,id,label
count,16000.000000,16000.000000
mean,7999.500000,7.512437
std,4618.946489,4.601214
min,0.000000,0.000000
25%,3999.750000,4.000000
50%,7999.500000,8.000000
75%,11999.250000,12.000000
max,15999.000000,15.000000


In [23]:
shape = {
    "image": (100, 75)
    # "image": (200, 150)
}
number_of_tests = 16000
data = np.zeros((number_of_tests, shape["image"][0], shape["image"][1]))

for id in labels['id']:
  data[id] = np.array(Image.open(join(trainDir, str(id) + ".tif")).resize((shape["image"][1], shape["image"][0])), dtype="uint")

In [24]:
# Convert the image matrix into linear data (i.e., 2D array into 1D array of features)
data_linear = np.array([x.reshape(np.prod(shape["image"])) for x in data])
data_linear.shape

(16000, 7500)

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

X_train, X_test, Y_train, Y_test = train_test_split(data_linear, labels["label"], test_size=0.25)

def logisticRegression(X, Y):
    file_name = "models/LogisticRegression.sav"
    if isfile(file_name):
        return joblib.load(file_name)

    pipe = Pipeline([
        ("scale", StandardScaler()),
        ("model", LogisticRegression(max_iter = 7500)),
    ])
    pipe.fit(X, Y)
    joblib.dump(pipe, file_name)
    
pipe = logisticRegression(X_train, Y_train)

Pipeline(steps=[('scale', StandardScaler()),
                ('model', LogisticRegression(max_iter=7500))])

In [41]:
correct = 0
Y_pred = pipe.predict(X_test)

correct_arr = filter(lambda x: x == 0, Y_pred - Y_test)
for i in correct_arr:
    correct += 1

# for i in range(len(Y_pred)):
#     if Y_pred[i] == Y_test[i]:
#         correct += 1
    
print("Total: ", len(Y_pred), ", Correct: ", correct)

Total:  4000 , Correct:  3377
